In [1]:
import os
import cv2 as cv
import numpy as np
import tensorflow as tf

In [2]:
# Train model
mnist = tf.keras.datasets.mnist

(pixel_train, class_train), (pixel_test, class_test) = mnist.load_data()

pixel_train = tf.keras.utils.normalize(pixel_train, axis = 1)
pixel_test = tf.keras.utils.normalize(pixel_test, axis = 1)


model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28,28)))
model.add(tf.keras.layers.Dense(128, activation = 'relu'))
model.add(tf.keras.layers.Dense(128, activation = 'relu'))
model.add(tf.keras.layers.Dense(10, activation = 'softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(pixel_train, class_train, epochs=5)

model.save('handwritten.model')

Epoch 1/5


2022-10-18 14:26:25.447948: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1875/1875 [==============================] - 2s 766us/step - loss: 0.2679 - accuracy: 0.9215
Epoch 2/5
1875/1875 [==============================] - 1s 767us/step - loss: 0.1111 - accuracy: 0.9659
Epoch 3/5
1875/1875 [==============================] - 1s 751us/step - loss: 0.0764 - accuracy: 0.9760
Epoch 4/5
1875/1875 [==============================] - 1s 756us/step - loss: 0.0568 - accuracy: 0.9818
Epoch 5/5
1875/1875 [==============================] - 1s 749us/step - loss: 0.0433 - accuracy: 0.9855
INFO:tensorflow:Assets written to: handwritten.model/assets


In [3]:
model = tf.keras.models.load_model('handwritten.model')

loss, accuracy = model.evaluate(pixel_test, class_test)

# print(loss)
# print(accuracy)

313/313 [==============================] - 0s 459us/step - loss: 0.0829 - accuracy: 0.9743


In [4]:
def rescaleFrame(frame, scale):
    width = int(frame.shape[1] * scale)
    height = int(frame.shape[1] * scale)
    print(width, height)
    dimensions = (width, height)
    
    return cv.resize(frame, dimensions, interpolation=cv.INTER_AREA)

In [5]:
capture = cv.VideoCapture(0)
scale = 0.50

while True:
    isTrue, frame = capture.read()
    frame_resized = rescaleFrame(frame, scale)

    # Grayscaled
    gray = cv.cvtColor(frame_resized, cv.COLOR_BGR2GRAY)
    gray_28 = cv.resize(gray, (28,28), interpolation=cv.INTER_AREA)
    
    gray_28 = gray_28.astype("float32") / 255.0
    gray_28 = np.expand_dims(gray_28, axis=-1)
    
    prediction = model.predict(gray_28.reshape(1,28,28))
    predargmax = np.argmax(prediction)

    # Black and white
    range = (150, 255)
    #range = (125, 255)
    ret, thresh = cv.threshold(gray, range[0], range[1], cv.THRESH_BINARY)
    contours, hierarchies = cv.findContours(thresh, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)

    # 28
    ret_28, thresh_28 = cv.threshold(gray, range[0], range[1], cv.THRESH_BINARY)
    thresh_28 = cv.resize(thresh_28, (28,28), interpolation=cv.INTER_AREA)
    contours_28, hierarchies_28 = cv.findContours(thresh_28, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
    thresh_28 = thresh_28.astype("float32") / 255.0
    thresh_28 = np.expand_dims(thresh_28, axis=-1)
    
    
    prediction_thresh_28 = model.predict(thresh_28.reshape(1,28,28))
    predargmax_thresh_28 = np.argmax(prediction_thresh_28)


    cv.putText(gray, 'Gray: ' + str(predargmax), (20,40), cv.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    cv.putText(gray, 'B&W: ' + str(predargmax_thresh_28), (20,80), cv.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 2)
    
    cv.imshow('Gray scale', gray)
    cv.imshow('Black & White (gray threshold)', thresh)
    cv.imshow('Black & White (gray threshold) 28x28', thresh_28)


    if cv.waitKey(20) & 0xFF==ord('q'):
        break

capture.release()
cv.destroyAllWindows()
cv.waitKey(20)

640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 9ms/step
640 640
1/1 [==============================] - 0s 9ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 9ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 8ms/step
640 640
1/1 [==============================] - 0s 8ms/st

-1

: 